# Lab 1b: Langchain Basics

In this lab, you will learn about LangChain — an open-source framework that helps developers build powerful applications using large language models (LLMs) like OpenAI's GPT. You'll start by installing the necessary packages, then explore chat models and basic tool usage, which will serve as a foundation for building AI agents in later labs.

## Installing LangChain
We'll start by installing the langchain package.

In [ ]:
!pip install langchain
!pip install langchain-openai

In [ ]:
# loading environment variables 
from dotenv import load_dotenv
load_dotenv(override=True)  # take environment variables

The most basic package in the Langchain ecosystem is langchain-core, which contains all classes and abstractions required to build other packages, except LangSmith package.  
<img src="https://python.langchain.com/assets/images/ecosystem_packages-32943b32657e7a187770c9b585f22a64.png" width="500">

## Learning about Messages
In Langchain, each message is defined by a role (e.g., "user", "assistant") and the content (e.g., text, multimodal data) with additional metadata that varies depending on the chat model provider. LangChain provides a unified message format that can be used across chat models, allowing users to work with different chat models without worrying about the specific details of the message format used by each model provider.

### Role 
Roles are used to distinguish between different types of messages in a conversation and help the chat model understand how to respond to a given sequence of messages.

| Role | Description |
|---|---|
|system|Used to tell the chat model how to behave and provide additional context. Not supported by all chat model providers.|
|user|Represents input from a user interacting with the model, usually in the form of text or other interactive input.|
|assistant|Represents a response from the model, which can include text or a request to invoke tools.|
|tool|A message used to pass the results of a tool invocation back to the model after external data or processing has been retrieved. Used with chat models that support tool calling.|

### Langchain Messages 
SystemMessage: corresponds to system role

HumanMessage: corresponds to user role

AIMessage: corresponds to assistant role

AIMessageChunk: corresponds to assistant role, used for streaming responses

ToolMessage: corresponds to tool role

RemoveMessage -- does not correspond to any role. This is an abstraction, mostly used in LangGraph to manage chat history.


In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, AIMessageChunk, RemoveMessage
import json 

human_message = HumanMessage("Hello, I am a human.")
system_message = SystemMessage("Hello, I am a system message.")
ai_message = AIMessage("Hello, I am an AI.")
ai_message_chunk = AIMessageChunk("I am a chunk of AI Message.")
remove_message = RemoveMessage(id="123")

print("Human Message")
print(human_message.model_dump_json(indent=2))

print("System Message")
print(system_message.model_dump_json(indent=2))

print("AI Message")
print(ai_message.model_dump_json(indent=2))

print("AI Message Chunk")
print(ai_message_chunk.model_dump_json(indent=2))

print("Removed Message")
print(remove_message.model_dump_json(indent=2))

## Prompt Templates
Prompt templates help to translate user input and parameters into instructions for a language model. This can be used to guide a model's response, helping it understand the context and generate relevant and coherent language-based output.

In [ ]:
# Simple Prompt Template

from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Berapa total penjualan produk {name}")

prompt_template.invoke({"name":"A"})

In [ ]:
# ChatPromptTemplates 
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate( [
    ("system", "You are a database expert and your task is to write a SQL statement based on a question from user. "
        "The SQL query statement shall be executed against an sqlite database."),
    ("user", "Berapa total penjualan produk {name}")
])

prompt_template.invoke({"name": "A"}).to_messages()

In [ ]:
# MessagesPlaceholder 
# Inserting the whole message instead of keywords.

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate(
    [("system", "You are a database expert and your task is to write a SQL statement based on a question from user. "
        "The SQL query statement shall be executed against an sqlite database."),
      MessagesPlaceholder("msg")
    ]
)

prompt_template.invoke({"msg": [SystemMessage(content='The database contains MthlySales table, with the following columns: ID, PRODUCT_NAME, SALES_QTY, SALES_AMOUNT, MONTH'), 
                                HumanMessage(content="Berapa total penjualan produk A")]}).to_messages()

In [ ]:
# Combine them

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate(
    [("system", "You are a database expert and your task is to write a SQL statement based on a question from user. "
        "The SQL query statement shall be executed against an sqlite database."),
      MessagesPlaceholder("msg"),
      ("user", "Berapa jumlah produk {name} yang terjual di bulan {month}")
    ]
)

prompt_template.invoke({"name":"A", 
                        "month":"3", 
                        "msg": [SystemMessage(content='The database contains MthlySales table, with the following columns: ID, PRODUCT_NAME, SALES_QTY, SALES_AMOUNT, MONTH')]
                        }).to_messages()

## Chat Models
Chat models are language models that use a sequence of messages as inputs and return messages as outputs (as opposed to using plain text). 

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4.1-mini", model_provider= "openai")
model.invoke("Please introduce yourself")

In [ ]:
# How to stream chat model responses

for chunk in model.stream("Write me a 1 verse song about goldfish on the moon"):
    print(chunk.content, end="", flush=True)

## Output Parsers
Using the .with_structured_output() method 

In [ ]:
# Pydantic Class 

from pydantic import BaseModel, Field

class ConceptList(BaseModel):
    """The list of concepts and brief description"""
    concept: str = Field(description="Important concept to be explained.")
    explanation: str = Field(description="Brief explanation of the concept")

structured_model = model.with_structured_output(ConceptList)

structured_model.invoke("I am learning about generative AI. Explain 1 random concept related to it.")

If you don't want to use Pydantic, explicitly don't want validation of the arguments, or want to be able to stream the model outputs, you can define your schema using a TypedDict class. We can optionally use a special Annotated syntax supported by LangChain that allows you to specify the default value and description of a field. Note, the default value is not filled in automatically if the model doesn't generate it, it is only used in defining the schema that is passed to the model.

Requirements

> Core: langchain-core>=0.2.26

> Typing extensions: It is highly recommended to import Annotated and TypedDict from typing_extensions instead of typing to ensure consistent behavior across Python versions.

In [ ]:
# TypedDict 

from typing_extensions import Annotated, TypedDict

class ConceptList(TypedDict):
    """The list of concepts and brief description"""
    concept: Annotated[str,..., "Important concept to be explained."] # type, default value, and description
    explanation: Annotated[str, ... , "Brief explanation of the concept"]

structured_model = model.with_structured_output(ConceptList)
structured_model.invoke("I am learning about generative AI. Explain 1 random concept related to it.")


In [ ]:
# JSON Schema
# Equivalently, we can pass in a JSON Schema dict. 
# This requires no imports or classes and makes it very clear exactly how each parameter is documented, 
# at the cost of being a bit more verbose.

json_schema = {
    "title": "ConceptList",
    "description": "The list of concepts and brief description",
    "type": "object",
    "properties": {
        "concept": {
            "type": "string",
            "description": "Important concept to be explained."
        },
        "explanation": {
            "type": "string",
            "description": "Brief explanation of the concept."
        }
    },
    "required": ["concept", "explanation"]
}

structured_model = model.with_structured_output(json_schema)
structured_model.invoke("I am learning about generative AI. Explain 1 random concept related to it.")


In [ ]:
# Choosing Multiple Schema

from pydantic import BaseModel, Field
from typing import Union

class ConceptList(BaseModel):
    """The list of concepts and brief description"""
    concept: str = Field(description="Important concept to be explained.")
    explanation: str = Field(description="Brief explanation of the concept")

class ConversationalResponse(BaseModel):
    """Respond in a conversational manner. Be kind and helpful."""

    response: str = Field(description="A conversational response to the user's query")


class FinalResponse(BaseModel):
    final_output: Union[ConceptList, ConversationalResponse]


structured_model = model.with_structured_output(FinalResponse)

structured_model.invoke("I am learning about generative AI. Explain 1 random concept related to it.")

In [ ]:
structured_model.invoke("How do you feel about generative AI technology?")

## Prompt Chaining

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

# Step 1: Set up the LLM
llm = init_chat_model("gpt-4.1-mini", model_provider= "openai", temperature = 0.7)
llm.invoke("Please introduce yourself")

In [ ]:
# Chain 1: Summarize input text
summarize_prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize the following article:\n\n{text}"
)

summarize_chain = summarize_prompt | llm

In [ ]:
input_text = """
Artificial Intelligence (AI) is transforming industries by enabling machines to learn from data, 
make decisions, and even improve over time. Applications range from chatbots and virtual assistants 
to complex data analytics and autonomous vehicles. However, AI also brings challenges such as ethical concerns, 
bias in algorithms, and job displacement. As AI continues to evolve, balancing innovation with responsible 
development will be key to its long-term success.
"""

full_response = ""
for chunk in summarize_chain.stream(input=input_text):
    full_response += chunk.content
    print(chunk.content, end="", flush=True)

In [ ]:
# Chain 2: Generate 3 quiz questions from the summary
question_prompt = PromptTemplate(
    input_variables=["summary"],
    template="Based on the summary below, generate 3 quiz questions:\n\n{summary}"
)
question_chain = question_prompt | llm

for chunk in question_chain.stream(input=full_response):
    full_response += chunk.content
    print(chunk.content, end="", flush=True)

In [ ]:
# Chain 3: Answer the first question
answer_prompt = PromptTemplate(
    input_variables=["questions"],
    template="Pick the first question from below and answer it in detail:\n\n{questions}"
)
answer_chain = answer_prompt | llm

for chunk in answer_chain.stream(input=full_response):
    full_response += chunk.content
    print(chunk.content, end="", flush=True)

In [ ]:
# Step 2: Compose full chain
full_chain = summarize_chain | question_chain | answer_chain

for chunk in full_chain.stream(input=input_text):
     print(chunk.content, end="", flush=True)